In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import DeepFMKit.core as dfm
from scipy.special import jv
from scipy.linalg import inv

def calculate_jacobian(ndata, param):
    """
    Calculates the Jacobian matrix (J) of the DFMI model.
    (This function remains unchanged from the previous version)
    """
    a, m, phi, psi = param
    J = np.zeros((2 * ndata, 4))
    j = np.arange(1, ndata + 1)
    
    phase_term = np.cos(phi + j * np.pi / 2.0)
    cos_jpsi = np.cos(j * psi)
    sin_jpsi = np.sin(j * psi)
    
    bessel_j = jv(j, m)
    bessel_deriv = 0.5 * (jv(j - 1, m) - jv(j + 1, m))
    
    common_term = a * phase_term * bessel_j
    model_q = common_term * cos_jpsi
    model_i = -common_term * sin_jpsi
    
    if a != 0:
        J[:ndata, 0] = model_q / a
        J[ndata:, 0] = model_i / a

    common_deriv_term_m = a * phase_term * bessel_deriv
    J[:ndata, 1] = common_deriv_term_m * cos_jpsi
    J[ndata:, 1] = -common_deriv_term_m * sin_jpsi

    phase_deriv_term = np.cos(phi + j * np.pi / 2.0 + np.pi / 2.0)
    common_deriv_term_phi = a * phase_deriv_term * bessel_j
    J[:ndata, 2] = common_deriv_term_phi * cos_jpsi
    J[ndata:, 2] = -common_deriv_term_phi * sin_jpsi

    J[:ndata, 3] = common_term * -sin_jpsi * j
    J[ndata:, 3] = -common_term * cos_jpsi * j
    
    return J

def calculate_m_precision(m_range, ndata, snr_db):
    """
    Core calculation function for statistical uncertainty of 'm'.
    
    Parameters
    ----------
    m_range : array_like
        The range of modulation depths 'm' to analyze.
    ndata : int
        Number of harmonics to use in the fit.
    snr_db : float
        Signal-to-Noise Ratio in dB for the I/Q measurements.

    Returns
    -------
    numpy.ndarray
        An array of the statistical uncertainty (delta_m) for each m in m_range.
    """
    param_fixed = np.array([1.0, 0, np.pi/4, 0.0]) # a, m, phi, psi
    snr_linear = 10**(snr_db / 20.0)
    noise_variance = (1.0 / snr_linear)**2
    
    delta_m_list = []
    for m_true in m_range:
        param_fixed[1] = m_true
        J = calculate_jacobian(ndata, param_fixed)
        JTJ = J.T @ J
        try:
            covariance_matrix = noise_variance * inv(JTJ)
            delta_m = np.sqrt(covariance_matrix[1, 1])
            delta_m_list.append(delta_m)
        except np.linalg.LinAlgError:
            delta_m_list.append(np.inf)
            
    return np.array(delta_m_list)

def validate_fitter_efficiency():
    """
    Performs a Monte Carlo simulation to test the NLS fitter's performance
    against the theoretical Cramér-Rao Lower Bound (CRLB).
    """
    # --- 1. Define Test Parameters ---
    # We choose a single, representative point in the parameter space.
    m_true = 15.5
    ndata = 15
    snr_db = 70.0
    
    # Simulation settings
    n_trials = 500  # Number of Monte Carlo runs. 500-1000 is a good number.
    n_fit_cycles = 20 # Number of modulation cycles per buffer

    # Derived parameters
    snr_linear = 10**(snr_db / 20.0)
    # The amplitude ASD is set to achieve the desired SNR.
    # Assuming signal amplitude is ~1, the noise RMS is 1/SNR.
    # ASD is related to RMS by sqrt(bandwidth). Here, bw is f_samp.
    # This is a simplification; a more rigorous calculation depends on the
    # processing gain, but this is sufficient for a consistent test.
    # Let's set amp=1 and calculate noise RMS in the buffer.
    amp_true = 1.0
    
    print("="*60)
    print("Fitter Efficiency Validation: Comparing Measured vs. Theoretical Precision")
    print(f"Parameters: m = {m_true}, ndata = {ndata}, SNR = {snr_db} dB")
    print(f"Number of Monte Carlo trials: {n_trials}")
    print("="*60)

    # --- 2. Calculate the Theoretical CRLB ---
    # Use the function we already built for this.
    delta_m_crlb = calculate_m_precision(np.array([m_true]), ndata, snr_db)[0]
    
    print(f"\nTheoretical Precision (CRLB): δm = {delta_m_crlb:.4e}")

    # --- 3. Run the Monte Carlo Simulation ---
    dff = dfm.DeepFitFramework()
    label = "efficiency_test"
    dff.new_sim(label)
    
    # Configure the simulation object
    sim_config = dff.sims[label]
    sim_config.m = m_true
    sim_config.amp = amp_true
    sim_config.f_mod = 1000
    sim_config.f_samp = 200000
    sim_config.fit_n = n_fit_cycles
    
    # IMPORTANT: We only enable amplitude noise to match the CRLB theory assumptions.
    # Calculate the required noise ASD.
    # Noise RMS = V_rms / SNR. For a sine wave, V_rms = A_peak / sqrt(2).
    # Noise RMS = (amp_true / np.sqrt(2)) / snr_linear
    # Let's assume the I/Q noise variance is (1/snr_linear)^2 as in analyze_precision.
    # And the time-domain voltage noise ASD that gives this is complex.
    # For simplicity, let's just set an amp_n and report the effective SNR.
    sim_config.amp_n = 1e-4 # A reasonable noise level

    m_estimates = []
    print("\nRunning Monte Carlo simulations...")
    
    # Initial guess for the fitter
    initial_guess = np.array([amp_true, m_true * 0.95, 0, 0]) # Start slightly off

    for i in tqdm(range(n_trials)):
        # Simulate a single buffer of data with a new noise realization
        # n_buffers=1 is key here.
        dff.simulate(label, n_buffers=1, simulate="static", trial_num=i)
        
        # We need to manually run the single-buffer fit logic
        raw_obj = dff.raws[label]
        R, fs, nbuf = dff.fit_init(label, n_fit_cycles)
        
        # Run the fit on the first (and only) buffer
        fit_result = dff._fit_single_buffer(label, 0, R, ndata, initial_guess)
        
        m_estimates.append(fit_result['m'])

    # --- 4. Analyze the Results ---
    m_estimates = np.array(m_estimates)
    delta_m_measured = np.std(m_estimates)
    
    # Calculate the efficiency of our estimator
    efficiency = (delta_m_crlb / delta_m_measured) * 100 if delta_m_measured > 0 else 0

    print("\n--- Results ---")
    print(f"Measured Precision (Std Dev of estimates): δm = {delta_m_measured:.4e}")
    print(f"Estimator Efficiency (CRLB / Measured): {efficiency:.1f}%")

    # --- 5. Plot the Distribution of Estimates ---
    plt.figure(figsize=(10, 6))
    plt.hist(m_estimates, bins=100, density=True, label='Distribution of $\hat{m}$', alpha=0.3)
    plt.axvline(m_true, color='k', linestyle='--', label=f'True m = {m_true}')
    plt.axvline(np.mean(m_estimates), color='r', linestyle=':', label=f'Mean $\hat{{m}}$ = {np.mean(m_estimates):.4f}')
    
    plt.title('Distribution of Estimated Modulation Depth (m)', fontsize=16)
    plt.xlabel('Estimated m value', fontsize=14)
    plt.ylabel('Probability Density', fontsize=14)
    plt.legend()
    plt.grid(True, linestyle=':')
    plt.tight_layout()
    plt.show()
    return m_estimates

if __name__ == '__main__':
    m_estimates = validate_fitter_efficiency()